# Task for the identification of argumentative / non-argumentative sentences

# Setup

Files needed to run the notebook:

- The pickle file *dataset.pkl*; 

Import libraries:

In [1]:
import pickle as pkl
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import tensorflow as tf
import os
import json
import numpy as np

from sentence_transformers import SentenceTransformer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.model_selection import train_test_split

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Dropout
from tensorflow.keras.regularizers import l2

from yellowbrick.text import FreqDistVisualizer


In [2]:
df = pd.read_pickle("./dataset.pkl")  

In [3]:
df.head()

,Document,Name,Id,Sentence,Type,Supported_by,Supported_from_failure,Attacked_by,Inhibited_by,Rephrased_by,Argumentation_scheme
2,2004,prem,A3,respect borne mind pursuant Article 58 Statute...,L,NaN,NaN,NaN,NaN,NaN,Rule
3,2004,prem,A4,Advocate General states point 20 Opinion Commi...,F,NaN,NaN,NaN,NaN,NaN,Aut
5,2004,prem,A6,Regarding Wam’s argument Commission’s appeal i...,L|F,NaN,NaN,NaN,NaN,NaN,Rule
6,2004,prem,A7,Appeals judgments Court First Instance governe...,L,NaN,NaN,NaN,NaN,NaN,Rule
7,2004,prem,A8,Next must noted obligation provide statement r...,L,NaN,NaN,NaN,NaN,NaN,Prec


In [4]:
df["Sentence"].values[0]

'respect borne mind pursuant Article 58 Statute Court Justice appeal Court Justice limited points law lie grounds particular infringement Community law Court First Instance'

In [5]:
df["Name"].value_counts()

prem    663
conc    110
Name: Name, dtype: int64

In [6]:
df_balanced = pd.concat([df[df["Name"] == "prem"].sample(110), df[df["Name"] =="conc"]])

In [7]:
df_balanced["Name"].value_counts()

prem    110
conc    110
Name: Name, dtype: int64

In [8]:
df_balanced.head()

,Document,Name,Id,Sentence,Type,Supported_by,Supported_from_failure,Attacked_by,Inhibited_by,Rephrased_by,Argumentation_scheme
1582,2028,prem,A13,general application derives fact decisions app...,L,NaN,NaN,NaN,NaN,NaN,Prec
48,2008,prem,A12,follows Member State relies test administrativ...,L,A11,NaN,NaN,NaN,NaN,Princ
1576,2028,prem,A7,remove certain kinds subcategories nonlegislat...,L,NaN,NaN,NaN,NaN,NaN,Itpr|Syst
257,2013,prem,D5,examination Commission whether particular meas...,L,NaN,NaN,NaN,NaN,NaN,Prec
1286,2027,prem,B3,particular competent authorities discretionary...,L,NaN,NaN,NaN,NaN,NaN,Itpr


Choice of parameters:

In [9]:
# EMBEDDINGS (uncomment the chosen one)
# EMBEDDING = "bert_sentence"
EMBEDDING = "legal_bert_sentence"
# EMBEDDING = "tfidf"

# MODELS (uncomment the chosen one)
MODEL = "SVC"
# MODEL = "RNN"
# MODEL = "BERT"

# DATASET (uncomment the chosen one)
# DATASET = df_balanced
DATASET = df

TEST_SIZE = 0.3
VAL_SIZE = 0.1  # percentage taken from the train subset

# MODEL PARAMETERS

# RNN & BERT
EPOCHS = 100
BATCH_SIZE = 64
LR = 0.0000001
DROP = 0.3

# ONLY RNN PARAMETERS
L2_FACTOR = 0.001
LAYERS = [128, 64, 32] # 3 layers


# SVC
KERNEL = "sigmoid" #{linear, poly, rbf, sigmoid’} 
GAMMA = 0.01
C =0.001
DEGREE = 3 # only for kernel "poly"

In [10]:
scheme_one_hot = pd.get_dummies(DATASET['Name'])

df_new = pd.concat([DATASET['Sentence'],scheme_one_hot],axis=1)
df_new.head()

,Sentence,conc,prem
2,respect borne mind pursuant Article 58 Statute...,0,1
3,Advocate General states point 20 Opinion Commi...,0,1
5,Regarding Wam’s argument Commission’s appeal i...,0,1
6,Appeals judgments Court First Instance governe...,0,1
7,Next must noted obligation provide statement r...,0,1


# Embedding

In [11]:
if EMBEDDING == "bert_sentence" or EMBEDDING == "legal_bert_sentence":

  if EMBEDDING == "bert_sentence":
    sbert_model = SentenceTransformer('bert-base-nli-mean-tokens')
  if EMBEDDING == "legal_bert_sentence":
    sbert_model = SentenceTransformer("nlpaueb/legal-bert-small-uncased")

  sentence_embeddings = sbert_model.encode(df_new["Sentence"].values)

  X = sentence_embeddings
  if MODEL == "RNN" or MODEL == "BERT":
    y = scheme_one_hot
  if MODEL == "SVC":
    y = DATASET['Name'].apply(lambda x: 1 if x=='prem' else 0)

Some weights of the model checkpoint at /root/.cache/torch/sentence_transformers/nlpaueb_legal-bert-small-uncased were not used when initializing BertModel: ['cls.predictions.transform.dense.weight', 'cls.predictions.decoder.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [12]:
if EMBEDDING == "tfidf":
  #Convert sentences into vectors 
  vectorizer = CountVectorizer()
  X = vectorizer.fit_transform(df_new["Sentence"].values)

  if MODEL == "RNN":
    y = scheme_one_hot
  if MODEL == "SVC":
    y = DATASET['Name'].apply(lambda x: 1 if x=='prem' else 0)

  # Embeddings with tfidf
  tfidf = TfidfTransformer()
  X = tfidf.fit_transform(X)
  X = X.toarray()

In [13]:
print("EMBEDDING: ", EMBEDDING)
print("SHAPE X: ", X.shape)
print("SHAPE y: ", y.shape)

EMBEDDING:  legal_bert_sentence
SHAPE X:  (773, 512)
SHAPE y:  (773,)


In [14]:
#Visualize word frequency
if EMBEDDING == "tfidf":
  features   = vectorizer.get_feature_names()
  visualizer = FreqDistVisualizer(features=features, size=(800, 1000))
  visualizer.fit(X)
  visualizer.finalize()

# Training

In [15]:
if MODEL == "RNN":
  OUTPUTS = y.shape[1]

  def create_deep_model(factor, rate):
      model = Sequential()      
      model.add(Dense(units=LAYERS[0],kernel_regularizer=l2(factor), 
        activation='relu')), Dropout(rate),
      model.add(Dense(units=LAYERS[1],kernel_regularizer=l2(factor),
        activation='relu')), Dropout(rate),
      model.add(Dense(units=LAYERS[2],kernel_regularizer=l2(factor),
        activation='relu')), Dropout(rate),
      #Output layer
      model.add(Dense(units=OUTPUTS, activation='softmax'))
      return model
  model= create_deep_model(factor=L2_FACTOR, rate=DROP)

  opt=tf.keras.optimizers.Adam(learning_rate=LR)
  model.compile(loss='categorical_crossentropy', optimizer=opt,   
  metrics=['accuracy'])

In [16]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=TEST_SIZE, shuffle= True)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=VAL_SIZE, shuffle= True)

if MODEL == "SVC":
  from sklearn import svm
  #fit the model
  model = svm.SVC(kernel=KERNEL, degree=DEGREE, gamma=GAMMA, coef0=C)
  model.fit(X_train, y_train)

if MODEL == "RNN":
  #fit the model
  history=model.fit(x=X_train, y=y_train, batch_size=BATCH_SIZE, epochs=EPOCHS, validation_data=(X_val, y_val), verbose=1)

if MODEL =="BERT":

  input = tf.keras.layers.Input(shape=(X_test.shape[-1]))
  drop = tf.keras.layers.Dropout(DROP, name="dropout")(input)
  output = tf.keras.layers.Dense(2, activation='relu', name="output")(drop)
  model = tf.keras.Model(inputs=[input], outputs = [output])
  
  opt=tf.keras.optimizers.Adam(learning_rate=LR)
  model.compile(loss='binary_crossentropy', optimizer=opt,   
  metrics=['accuracy'])
  history=model.fit(x=X_train, y=y_train, batch_size=BATCH_SIZE, epochs=EPOCHS, validation_data=(X_val, y_val), verbose=1)

# Evaluation

In [17]:
if MODEL == "RNN":
  y_test=y_test.values
  y_test=np.argmax(y_test, axis=1)
  y_pred = np.argmax(model.predict(X_test), axis=-1)
  from sklearn.metrics import classification_report
  print(classification_report(y_test, y_pred, zero_division=0, target_names=df_new.columns[1:]))

if MODEL == "SVC":
  y_test=y_test.values
  #y_test=np.argmax(y_test, axis=1)
  y_pred = model.predict(X_test)
  from sklearn.metrics import classification_report
  print(classification_report(y_test, y_pred, zero_division=0, target_names=df_new.columns[1:]))

if MODEL =="BERT":

  y_test= y_test.values
  y_test=np.argmax(y_test, axis=1)
  y_pred = np.argmax(model.predict(X_test), axis=-1)
  from sklearn.metrics import classification_report
  print(classification_report(y_test, y_pred, zero_division=0, target_names=df_new.columns[1:]))

              precision    recall  f1-score   support

        conc       1.00      0.94      0.97        36
        prem       0.99      1.00      0.99       196

    accuracy                           0.99       232
   macro avg       0.99      0.97      0.98       232
weighted avg       0.99      0.99      0.99       232



In [18]:
y_pred

array([0, 1, 0, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1,
       1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 0, 0, 0, 1, 1, 1, 0, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1,
       0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1,
       0, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1])

Saving of test parameters and results on file:

In [19]:
with open("prem_conc_classification_results.txt", "a") as f:
    f.write("MODEL: " + MODEL + "\n")
    f.write("EMBEDDING: " + EMBEDDING  + "\n")
    f.write("\n")

    f.write("DATASET:\n")
    f.write(str(DATASET["Name"].value_counts()))
    f.write("\n\n")
    f.write("NUMBER OF SAMPLES: " + str(len(X)) + "\n")
    f.write("TEST_SIZE: " + str(TEST_SIZE*100) + "%\n")
    f.write("\n")

    f.write("PARAMETERS:\n")

    if MODEL == "RNN":
      f.write("EPOCHS: " + str(EPOCHS) + "\n")
      f.write("BATCH_SIZE: " + str(BATCH_SIZE) + "\n")
      f.write("LAYERS: " + str(LAYERS) + "\n")
      f.write("LR: " + str(LR) + "\n")
      f.write("L2_FACTOR: " + str(L2_FACTOR) + "\n")
      f.write("DROP: " + str(DROP) + "\n")
      f.write("\n")

    if MODEL == "BERT":
      f.write("EPOCHS: " + str(EPOCHS) + "\n")
      f.write("BATCH_SIZE: " + str(BATCH_SIZE) + "\n")
      f.write("LR: " + str(LR) + "\n")
      f.write("DROP: " + str(DROP) + "\n")
      f.write("\n")

    if MODEL == "SVC":
      f.write("KERNEL: " + str(KERNEL) + "\n")
      f.write("GAMMA: " + str(GAMMA) + "\n")
      f.write("C: " + str(C) + "\n")
      f.write("DEGREE: " + str(DEGREE) + "\n")
      f.write("\n")

    f.write("RESULTS:\n")
    f.write(classification_report(y_test, y_pred, zero_division=0, target_names=df_new.columns[1:])+"\n")

    f.write("\n")
    f.write("#######################################################")
    f.write("\n\n")
    

